## Rossmann case - Forecast Sales analysis
Olá! Seja bem-vindo(a)!

Este projeto foi desenvolvido com foco no desenvolvimento do meu portfólio de análise de dados, demonstrando minhas habilidades em Análise Exploratória de Dados com Python e utilização de estatística e regressão linear para previsão de demanda.

### Contexto de negócio
A Rossmann é uma das maiores redes de farmacias da Europa, com mais de 3.000 lojas em operação. 
Para planejar um ciclo de reformas, o CFO da empresa precisa saber com antecedência quanto cada loja vai faturar nas próximas seis semanas, informação essencial para uma alocação de orçamento inteligente. O objetivo deste projeto é construir um modelo de regressão linear que faça essa previsão com base em variáveis como promoções, feriados, sazonalidade e presença de concorrentes.

### Perguntas de negócio a serem respondidas
    1. Quanto cada loja vai vender nas próximas 6 semanas?

### Etapas da Análise

- Importação de bibliotecas e carregamento dos dados
- Limpeza, tratamento e preparação
- Análise Exploratória de Dados (EDA)
- Preparação dos dados
- Modelagem com Regressão Linear
- Conclusões Regressão
- Conclusões Gerais


Fique à vontade para entrar em contato comigo. Boa leitura!

---

### 📖 Carregamento das bibliotecas.

In [3]:
import pandas as pd

### 📥 Importação de dados

In [4]:
df_train = pd.read_csv('../sales_forecast_rossmann/rossmann-store-sales-files/train.csv', sep=',')
df_train

C:\Users\diogo\AppData\Local\Temp\ipykernel_24752\4023907617.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv('../sales_forecast_rossmann/rossmann-store-sales-files/train.csv', sep=',')


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1
...,...,...,...,...,...,...,...,...,...
1017204,1111,2,2013-01-01,0,0,0,0,a,1
1017205,1112,2,2013-01-01,0,0,0,0,a,1
1017206,1113,2,2013-01-01,0,0,0,0,a,1
1017207,1114,2,2013-01-01,0,0,0,0,a,1


### 🧹Limpeza, tratamento e preparação
1. Valores ausentes.
2. Valores duplicados.
3. Tipo de dados.
4. Outliers.

In [ ]:
#1 > ok
df_train.isnull().sum()
#2 > ok
df_train.duplicated().sum()

Store            0
DayOfWeek        0
Date             0
Sales            0
Customers        0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64

In [27]:
#2 > ok
df_train['Date'] = pd.to_datetime(df_train['Date'])
df_train['StateHoliday'] = df_train['StateHoliday'].replace({'0':'0','a':'1','b':'1','c':'1'})
df_train['StateHoliday'] = df_train['StateHoliday'].astype(int)
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   Store          1017209 non-null  int64         
 1   DayOfWeek      1017209 non-null  int64         
 2   Date           1017209 non-null  datetime64[ns]
 3   Sales          1017209 non-null  int64         
 4   Customers      1017209 non-null  int64         
 5   Open           1017209 non-null  int64         
 6   Promo          1017209 non-null  int64         
 7   StateHoliday   1017209 non-null  int64         
 8   SchoolHoliday  1017209 non-null  int64         
dtypes: datetime64[ns](1), int64(8)
memory usage: 69.8 MB


In [56]:
df_train = df_train[df_train['Sales'] != 0]
sales_by_client = df_train.groupby('Customers')['Sales'].sum().describe().astype(int)
sales_by_client
#Identifiquei uma distribuição bem assimetrica, portanto,
#há necessidade de remoção de Outliers para que não afete as projeções 



count        4083
mean      1438447
std       2511721
min            46
25%         76353
50%        218009
75%       1451833
max      14597326
Name: Sales, dtype: int64

In [ ]:
#Remoção de outliers superiores com base no IQR
#(Aqui se em analise detalhada, entender-se que essas vendas não são "Casuais", poderiamos manter)
Q1 = sales_by_client.quantile(0.25)
Q3 = sales_by_client.quantile(0.75)
IQR = Q3 - Q1


#realizei um primeiro teste com a remoção de outlier padrão, conforme abaixo,
#mas como caracteristica dessa base de dados, é uma grande assimetria dos dados
#portanto fez mais sentido colocar uma suavização na remoção, multiplicando por 3
#upper_limit = Q3 + 1.5 * IQR
#lower_limit = Q1 - 1.5 * IQR

upper_limit = Q3 + 1.5 * IQR
lower_limit = Q1 - 1.5 * IQR #Não irá filtrar nada pois o resultado é negativo

valid_clients = sales_by_client[
    (sales_by_client >= lower_limit) &
    (sales_by_client <= upper_limit)
].index

df_train = df_train[df_train['Customers'].isin(valid_clients)]
df_train.head()
#REVISAR, MESMO MUDANDO REGRAS ENTRE PERCENTIL E 1.5 QTD LINHAS NAO MUDA